## Generate VEX function completions

Finally, verified signatures documentation in Houdini 16.5!
I'll discard all previous work I did in last couple of years
and will rely on the docs.

In [1]:
import sys
import pathlib
import zipfile
import json

from hfs import HFS
sys.path.append(str(HFS / 'houdini' / 'python3.9libs'))
from bookish.wiki.wikipages import parse_to_root as parse_wikipage
from houdinihelp import vex

In [2]:
def extract_text(node):
    '''Evaluate possibly nested 'text' element to string.'''
    if isinstance(node, dict):
        return extract_text(node['text'])
    elif isinstance(node, list):
        return ''.join(extract_text(i) for i in node)
    else:
        return node


def usages(node):
    '''Recursively extract all 'type' values from Bookish tree.'''
    if type(node) is dict:
        if node['type'] == 'usage':
            yield extract_text(node['text'])

        for k in node:
            if type(node[k]) is list:
                for i in node[k]:
                    for j in usages(i):
                        yield j


def all_usages(functions):
    '''Convenience loop for many Bookish trees stored in a dict.'''
    for f in functions:
        for u in usages(functions[f]):
            yield u

In [3]:
# Open vex.zip containing various helpcards including functions.
# Parse Wiki files into JSON using Bookish wiki parser.
docs = {}

with zipfile.ZipFile(HFS / 'houdini' / 'help' / 'vex.zip') as z:
    for path in z.namelist():
        path = pathlib.PurePath(path)
        if path.parent.stem == 'functions':
            with z.open(path.as_posix()) as f:
                name = path.stem
                markup = f.read().decode()
                docs[name] = parse_wikipage(markup)

# Parse function usage examples from docs.
parsed_usages = [vex.parse_vex(usage) for usage in all_usages(docs)]
len(parsed_usages)

2197

In [4]:
comps = {
    'scope': 'source.vex -string -comment -source.hscript',
    'completions': []
}
unique_triggers = set()

for usage in parsed_usages:
    function_name = usage.ident.name

    args = []
    for arg in usage.arglist.args:
        if isinstance(arg, vex.VariadicArgs):
            args.append(arg.string())
        else:
            args.append(arg.ident.string())

    trigger = f"{function_name}({', '.join(args)})"

    # Completion exists.
    if trigger in unique_triggers:
        continue
        
    # Completion not exists. Make a new one.
    unique_triggers.add(trigger)

    # Build contents.
    cargs = [f'${{{i}:{arg}}}' for i, arg in enumerate(args, 1)]
    contents = f"{function_name}({', '.join(cargs)})"

    comps['completions'].append({'trigger': trigger,
                                 'contents': contents})


print('Generated %d completions.' % len(unique_triggers))


# Write completions into a functions.sublime-completions file.
with open('functions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 1691 completions.
